In [ ]:
class MockLogger:
    def debug(self, message):
        print(f"DEBUG: {message}")

logger = MockLogger()

class Response:
    def __init__(self, text):
        self.text = text

response = Response("This is a long response from the API that contains lots of text and goes beyond 200 characters to show how the truncation works in the debug message when logging the response content.")

print("Testing the code:")
if hasattr(response, 'text') and response.text:
    logger.debug(f"Gemini response received: {response.text[:200]}...")
    result = response.text
    print(f"Returned: {result[:50]}...")


In [1]:
import os
import logging
from dotenv import load_dotenv
import google.generativeai as genai
from abc import ABC, abstractmethod

load_dotenv()

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

/Users/mahmoud/agent-with-graphiti/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LLMProvider(ABC):
    @abstractmethod
    def generate(self, prompt: str) -> str:
        pass

class GeminiProvider(LLMProvider):
    def __init__(self, model_name: str = "gemini-2.5-flash"):
        genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
        self.model = genai.GenerativeModel(model_name)

    def generate(self, prompt: str) -> str:
        logger.debug(f"Generating response for prompt length: {len(prompt)}")
        max_retries = 3
        for attempt in range(max_retries):
            try:
                logger.debug(f"Gemini API call attempt {attempt + 1}")
                response = self.model.generate_content(
                    prompt,
                    generation_config=genai.types.GenerationConfig(
                        temperature=0,
                        max_output_tokens=500,
                        top_p=0,
                        top_k=40
                    )
                )

                if hasattr(response, 'text') and response.text:
                    logger.debug(f"Gemini response received: {response.text[:200]}...")
                    return response.text

                logger.warning(f"Empty response from Gemini (attempt {attempt + 1})")

            except Exception as e:
                logger.error(f"Gemini API error (attempt {attempt + 1}): {e}")
                if attempt == max_retries - 1:
                    fallback_response = '{"thought": "Technical difficulties", "answer": "I\'m experiencing technical issues. Please try again."}'
                    logger.debug(f"Returning fallback response: {fallback_response}")
                    return fallback_response

                import time
                time.sleep(1)

        fallback_response = '{"thought": "Max retries exceeded", "answer": "Connection issues. Please try again later."}'
        logger.debug(f"Max retries exceeded, returning: {fallback_response}")
        return fallback_response

In [3]:
# Simple test template for ReAct agent
def create_test_prompt(query: str, history: str = "") -> str:
    return f"""You are an expert Customer Service Agent. Analyze the user's query to understand their intent and plan the appropriate response with the provided tools.
Query: {query}
History: {history}
Available tools: SEARCH_PRODUCTS, PLACE_ORDER, GET_ORDER_HISTORY

For SEARCH_PRODUCTS tool, use separate attributes:
- "input": user input
- "color": color if specified
- "model": model/variant if specified
Respond in JSON format:
{{"thought": "reasoning", "action": {{"name": "SEARCH_PRODUCTS", "input": "iPhone", "color": "red", "model": "15 Pro"}}}}
OR
{{"thought": "reasoning", "answer": "final answer"}}
"""

llm_provider = GeminiProvider()

prompt1 = create_test_prompt("I want to bur latest iPhone")
print(f"PROMPT:\n{prompt1}")
print("\n" + "-" * 30)

response1 = llm_provider.generate(prompt1)
print(f"JSON RESPONSE:\n{response1}")



2025-07-10 18:12:27,523 - DEBUG - Generating response for prompt length: 601
2025-07-10 18:12:27,524 - DEBUG - Gemini API call attempt 1


PROMPT:
You are an expert Customer Service Agent. Analyze the user's query to understand their intent and plan the appropriate response with the provided tools.
Query: I want to bur latest iPhone
History: 
Available tools: SEARCH_PRODUCTS, PLACE_ORDER, GET_ORDER_HISTORY

For SEARCH_PRODUCTS tool, use separate attributes:
- "input": user input
- "color": color if specified
- "model": model/variant if specified
Respond in JSON format:
{"thought": "reasoning", "action": {"name": "SEARCH_PRODUCTS", "input": "iPhone", "color": "red", "model": "15 Pro"}}
OR
{"thought": "reasoning", "answer": "final answer"}


------------------------------


2025-07-10 18:12:31,309 - DEBUG - Gemini response received: ```json
{
  "thought": "The user wants to 'buy' (likely a typo for 'bur') the 'latest iPhone'. To fulfill this request, I need to search for iPhones. The `SEARCH_PRODUCTS` tool is appropriate for this...


JSON RESPONSE:
```json
{
  "thought": "The user wants to 'buy' (likely a typo for 'bur') the 'latest iPhone'. To fulfill this request, I need to search for iPhones. The `SEARCH_PRODUCTS` tool is appropriate for this. Since the user specified 'latest' but no specific model number or color, I will search for 'iPhone' generally. The system or a subsequent step would then identify the 'latest' model from the search results.",
  "action": {
    "name": "SEARCH_PRODUCTS",
    "input": "iPhone"
  }
}
```


In [5]:

# history = "user: I want to search for iPhone\nassistant: " + response1
prompt2 = create_test_prompt("15 Pro in red color")

response2 = llm_provider.generate(prompt2)
print(f"JSON RESPONSE:\n{response2}")

# print("\n" + "=" * 50)
# print("=" * 50)
# print(f"Response 1: {response1}")
# print(f"Response 2: {response2}")

2025-07-10 18:13:32,845 - DEBUG - Generating response for prompt length: 593
2025-07-10 18:13:32,846 - DEBUG - Gemini API call attempt 1
2025-07-10 18:13:34,591 - DEBUG - Gemini response received: ```json
{
  "thought": "The user is looking for a specific product, '15 Pro' with a 'red' color. The `SEARCH_PRODUCTS` tool is appropriate for this. I can extract '15 Pro' as the model and 'red' as th...


JSON RESPONSE:
```json
{
  "thought": "The user is looking for a specific product, '15 Pro' with a 'red' color. The `SEARCH_PRODUCTS` tool is appropriate for this. I can extract '15 Pro' as the model and 'red' as the color. Since '15 Pro' is commonly associated with iPhone, I will use 'iPhone' as the general input.",
  "action": {
    "name": "SEARCH_PRODUCTS",
    "input": "iPhone",
    "color": "red",
    "model": "15 Pro"
  }
}
```
